In [13]:
import tensorflow as tf
from transformers import AutoTokenizer
from transformers import TFGPT2LMHeadModel #tensorflow GPT2 LM
import numpy as np
import torch

In [23]:
tf.config.list_physical_devices('GPU') 

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:3', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:4', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:5', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:6', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:7', device_type='GPU')]

: 

In [11]:
mirrored_strategy = tf.distribute.MirroredStrategy()

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [15]:
model = TFGPT2LMHeadModel.from_pretrained('skt/kogpt2-base-v2', from_pt=True)
tokenizer = AutoTokenizer.from_pretrained('skt/kogpt2-base-v2')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFGPT2LMHeadModel: ['transformer.h.5.attn.masked_bias', 'transformer.h.11.attn.masked_bias', 'transformer.h.10.attn.masked_bias', 'transformer.h.2.attn.masked_bias', 'lm_head.weight', 'transformer.h.4.attn.masked_bias', 'transformer.h.0.attn.masked_bias', 'transformer.h.1.attn.masked_bias', 'transformer.h.7.attn.masked_bias', 'transformer.h.6.attn.masked_bias', 'transformer.h.3.attn.masked_bias', 'transformer.h.9.attn.masked_bias', 'transformer.h.8.attn.masked_bias']
- This IS expected if you are initializing TFGPT2LMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFGPT2LMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassifica

In [16]:
#실제 모델에서 제공하는 generate (추론 과정))
sent = '아이들의 정서적인 발달에 동화책은'
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
with mirrored_strategy.scope():
    output = model.generate(input_ids,
                        max_length=200,
                        repetition_penalty=2.0,
                        use_cache=False)

#=====여기부터 워터마킹======

In [17]:
output_ids = output.numpy().tolist()[0]
tokenizer.decode(output_ids)

'아이들의 정서적인 발달에 동화책은 큰 도움이 될 수 있다.\n또한 책읽는 습관을 길러주는 효과도 기대할 만하다.\n책 읽기를 통해 아이들은 자연스럽게 책을 읽을 기회를 갖게 된다.\n책을 읽는 동안 아이는 스스로에게 질문을 던지고 답을 찾는 과정을 반복하게 되고, 그 과정에서 아이가 어떤 생각을 하게 되는지 알게 되면서, 자신의 생각과 느낌을 글로 표현하는 능력을 키울 수도 있게 된다.</d> 지난해 12월 31일 오후 2시쯤 서울 강남구 역삼동 한 아파트에서 A(40)씨가 숨진 채 발견됐다.\nA씨는 이날 오전 9시경 경찰 조사 도중 “아파트 엘리베이터 안에서 목을 매 자살했다”고 진술했다.\n경찰은 유서가 발견되지 않은 점 등으로 미뤄 정확한 사인을 밝히기 위해 부검을 의뢰할 방침이다.\n유서는 없었다.\n숨지기 전날인 30일에는 경기도 용인시에서 택시기사 B씨(47)가 술에 취해 귀가하던 중 흉기에 찔려 사망했다.\nB씨의 시신은 인근 병원으로 옮겨졌으나 끝내 숨을 거뒀다.\n이날 새벽 1시부터 자정까지 경기 수원시의 CCTV'

In [18]:
sent = '아이들의 정서적인 발달에 동화책은'

In [19]:
input_ids = tokenizer.encode(sent)
input_ids = tf.convert_to_tensor([input_ids])
print(input_ids)

tf.Tensor([[42738 17381  9090 24004 19383 16095]], shape=(1, 6), dtype=int32)


In [20]:
#output = model.generate(input_ids,
#                        max_length=128,
#                        repetition_penalty=2.0,
#                        use_cache=True)

output = model(input_ids)
Logits = output.logits
Logits

<tf.Tensor: shape=(1, 6, 51200), dtype=float32, numpy=
array([[[-6.3650374 , -5.9244194 , -5.5607867 , ...,  0.78603697,
         -5.022273  , -2.9620056 ],
        [-7.120081  , -7.1628304 , -6.602664  , ..., -2.2832334 ,
         -5.9630513 , -4.78648   ],
        [-7.444871  , -6.6141295 , -4.999629  , ...,  0.51350355,
         -2.3515596 , -4.5314565 ],
        [-5.8011975 , -6.3517947 , -5.1028595 , ...,  2.6300447 ,
         -0.4167254 , -1.6408297 ],
        [-6.1230216 , -6.6718645 , -5.732671  , ...,  1.2544551 ,
         -0.70743334, -2.309436  ],
        [-5.313277  , -5.550266  , -5.276083  , ...,  5.036843  ,
          0.1513547 , -2.1300857 ]]], dtype=float32)>

In [21]:
Logits.shape

TensorShape([1, 6, 51200])

In [ ]:

with mirrored_strategy.scope(): # Logit 계속 받아오는거랑 generate 랑 같은 지 보기
    while input_ids.shape[-1] < 100:
     
        print("================================")
        print(input_ids.shape[-1])
        Logits = model(input_ids).logits
        
        add_token_id = tf.argmax(Logits)
        add_token_id = tf.reshape(add_token_id, [1,1])
        add_token_id = tf.cast(add_token_id, input_ids.dtype)
        input_ids = tf.concat([input_ids, add_token_id], axis=-1)
        
        next_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(next_token)
    
        


In [22]:

last_index = input_ids.shape[-1]

with mirrored_strategy.scope():
    while input_ids.shape[-1] < 100:
        print("================================")
        print(input_ids.shape[-1])
        #set 나눠서 green set 로짓을 촉진하기
        gamma = 1.0
        delta = 0.0
        Logits = model(input_ids).logits
        print(f"original Logits : {Logits}")

        #seed = int(hash(input_ids % 51200))
        seed = input_ids[-1] % Logits.shape[-1]
        print(f"seed : {seed[-1]}")
        np.random.seed(seed[-1])

        indices = np.arange(Logits.shape[-1]) #0~51999
        green_list_size = int(Logits.shape[-1] * gamma)  #25600
        green_list = np.random.choice(indices, green_list_size, replace=False)
        print(f"green list : {green_list} {green_list.shape}") #25600개의 green list

        #그린 리스트에 속한 idx 값의 Logit에는 델타 추가!!!!!!
        # 인덱스와 업데이트를 위한 텐서 생성
        print(f"last index = last words : {last_index}")
        
        #--여기부터 probability 구하기
        
        green_logits = tf.zeros(green_list_size)
        red_logits = tf.zeros(Logits.shape[-1] - green_list_size)
        for i in green_list:
            indices = [[0, last_index - 1, i]]  # 차원에 맞는 인덱스 설정
            updates = [Logits[0, last_index - 1, i] + delta]  # 업데이트할 값
            Logits = tf.tensor_scatter_nd_update(Logits, indices, updates)

        print(f"updated Logits : {Logits}")
        
        denominator = tf.reduce_sum(tf.exp(Logits), axis=-1)  # softmax 계산을 위한 분모
        print(f"dominants : {denominator}") #25600개의 green list

        # Logits 텐서에 softmax 적용하여 확률 계산
        softmax_tensor = tf.nn.softmax(Logits[0, last_index - 1, :])
        print(softmax_tensor)
        print(softmax_tensor.shape)


        print(f"make Probabilities : {softmax_tensor}")

        # 데이터 타입 확인 및 조정
        add_token_id = tf.argmax(softmax_tensor)
        print(f"updated Logits Max value : {add_token_id}")


        # 텐서 연결
        add_token_id = tf.reshape(add_token_id, [1,1])
        add_token_id = tf.cast(add_token_id, input_ids.dtype)
        input_ids = tf.concat([input_ids, add_token_id], axis=-1)
        next_token = tokenizer.decode(input_ids.numpy()[0,:])
        print(f"next token : {next_token}")
        
        last_index = input_ids.shape[-1]

6
original Logits : [[[-6.3650374  -5.9244194  -5.5607867  ...  0.78603697 -5.022273
   -2.9620056 ]
  [-7.120081   -7.1628304  -6.602664   ... -2.2832334  -5.9630513
   -4.78648   ]
  [-7.444871   -6.6141295  -4.999629   ...  0.51350355 -2.3515596
   -4.5314565 ]
  [-5.8011975  -6.3517947  -5.1028595  ...  2.6300447  -0.4167254
   -1.6408297 ]
  [-6.1230216  -6.6718645  -5.732671   ...  1.2544551  -0.70743334
   -2.309436  ]
  [-5.313277   -5.550266   -5.276083   ...  5.036843    0.1513547
   -2.1300857 ]]]
seed : 16095
green list : [33079 19547 38315 ... 42505 30137 32538] (51200,)
last index = last words : 6
updated Logits : [[[-6.3650374  -5.9244194  -5.5607867  ...  0.78603697 -5.022273
   -2.9620056 ]
  [-7.120081   -7.1628304  -6.602664   ... -2.2832334  -5.9630513
   -4.78648   ]
  [-7.444871   -6.6141295  -4.999629   ...  0.51350355 -2.3515596
   -4.5314565 ]
  [-5.8011975  -6.3517947  -5.1028595  ...  2.6300447  -0.4167254
   -1.6408297 ]
  [-6.1230216  -6.6718645  -5.732671 

KeyboardInterrupt: 

In [ ]:
#top5 = tf.math.top_k(output.logits[0, -1], k=5)